In [1]:
# Import dependencies
import pandas as pd
from pathlib import Path
import chardet
import matplotlib.pyplot as plt
import scipy.stats as st
import numpy as np

In [4]:
# Read in cleaned CSV
merged_data = pd.read_csv('Cleaned_Dataframes/Merged_Dataframes_cleaned.csv')
merged_data.head()

,Unnamed: 0,State,Percentage of Uninsured Adults,Percentage of Adults visit Reg Doctors,Avoidable Deaths per 100k Population,Treatable Deaths per 100k Population,Preventable Deaths per 100k Population
0,0,Alabama,15.25,77.65,321.2,108.5,212.6
1,1,Alaska,15.60,65.55,261.7,67.9,193.9
2,2,Arizona,15.00,72.20,249.3,76.1,173.2
3,3,Arkansas,12.70,80.00,351.0,121.7,229.3
4,4,California,10.55,74.65,203.8,70.8,133.1


In [ ]:
# Copying describe data - NEEDS EDITED
unis_clean_1819.describe()
avoi_deaths_df.describe()
prev_deaths_df.describe()
treat_deaths_df.describe()

In [ ]:
# Copying - NEEDS EDITED
variance_unis_clean = unis_clean_1819['Percentage of Uninsured Adults'].var()
variance_unis_clean

In [6]:
# Copying - NEEDS EDITED
std_deviation_unis_clean = unis_clean_1819['Percentage of Uninsured Adults'].std()
std_deviation_unis_clean

In [ ]:
# Copying - NEEDS EDITED
x_axis = np.arange(len(avoi_deaths_df))
tick_locations = [State for State in x_axis]
plt.figure(figsize=(20,4))
plt.bar(x_axis, avoi_deaths_df["Deaths per 100k Population"], color='b', alpha=0.5, align="center")
plt.xticks(tick_locations, avoi_deaths_df["State"], rotation="vertical")
plt.show()

In [ ]:
# COPY - NEEDS EDITED
# This one is NOT correct yet - playing with the idea of standard deviation bars - currently it's not doing standard deviation from the average
# std_dev = avoi_deaths_df['Deaths per 100k Population'].std()

# x_axis = np.arange(len(avoi_deaths_df))
# tick_locations = [State for State in x_axis]
# plt.figure(figsize=(20,4))
# plt.bar(x_axis, avoi_deaths_df["Deaths per 100k Population"], color='b', alpha=0.5, align="center")
# plt.xticks(tick_locations, avoi_deaths_df["State"], rotation="vertical")
# plt.errorbar(avoi_deaths_df['State'], avoi_deaths_df['Deaths per 100k Population'], yerr=std_dev, fmt='o', color='r', ecolor='black', capsize=5)
# plt.show()

In [ ]:
#COPY _ EDITS NEEDED
# Sort the data set high to low - Top 5 results
unis_clean_1819_sorted = unis_clean_1819.sort_values("Percentage of Uninsured Adults")
unis_clean_1819_sorted.head()

In [ ]:
# COPY
# bottom 5 results
unis_clean_1819_sorted.tail()

In [ ]:
# COPY
x_axis = np.arange(len(unis_clean_1819_sorted))
tick_locations = [State for State in x_axis]
plt.figure(figsize=(20,4))
plt.bar(x_axis, unis_clean_1819_sorted["Percentage of Uninsured Adults"], color='b', alpha=0.5, align="center")
plt.xticks(tick_locations, unis_clean_1819_sorted["State"], rotation="vertical")
plt.title('Average Percentage of Uninsured Individuals per State (2018-2019)')
plt.xlabel('State')
plt.ylabel('Percentage of Uninsured Individuals')
plt.show()

In [ ]:
# COPY 
# Sort the data set high to low - top 5 results
avoi_deaths_df_sorted = avoi_deaths_df.sort_values("Deaths per 100k Population")
avoi_deaths_df_sorted.head()

In [ ]:
# COPY
# Bottom 5 results
avoi_deaths_df_sorted.tail()

In [ ]:
# COPY
x_axis = np.arange(len(avoi_deaths_df_sorted))
tick_locations = [State for State in x_axis]
plt.figure(figsize=(20,4))
plt.bar(x_axis, avoi_deaths_df_sorted["Deaths per 100k Population"], color='b', alpha=0.5, align="center")
plt.xticks(tick_locations, avoi_deaths_df_sorted["State"], rotation="vertical")
plt.title('Avoidable Deaths per 100,000 Population per State (2018-2019)')
plt.xlabel('State')
plt.ylabel('Deaths per 100k Population')
plt.show()

In [ ]:
# COPY
# Sort the data set high to low 
treat_deaths_df_sorted = treat_deaths_df.sort_values("Deaths per 100k Population")
treat_deaths_df_sorted.head()

In [ ]:
# COPY
# Bottom 5 results
treat_deaths_df_sorted.tail()

In [ ]:
# COPY
x_axis = np.arange(len(treat_deaths_df_sorted))
tick_locations = [State for State in x_axis]
plt.figure(figsize=(20,4))
plt.bar(x_axis, treat_deaths_df_sorted["Deaths per 100k Population"], color='g', alpha=0.5, align="center")
plt.xticks(tick_locations, treat_deaths_df_sorted["State"], rotation="vertical")
plt.title('Treatable Deaths per 100,000 Population per State (2018-2019)')
plt.xlabel('State')
plt.ylabel('Deaths per 100k Population')
plt.show()

In [ ]:
# COPY
# Sort the data set high to low - top 5 results
prev_deaths_df_sorted = prev_deaths_df.sort_values("Deaths per 100k Population")
prev_deaths_df_sorted.head()

In [ ]:
# COPY
# Bottom 5 results
prev_deaths_df_sorted.tail()

In [ ]:
# COPY
x_axis = np.arange(len(prev_deaths_df_sorted))
tick_locations = [State for State in x_axis]
plt.figure(figsize=(20,4))
plt.bar(x_axis, prev_deaths_df_sorted["Deaths per 100k Population"], color='y', alpha=0.5, align="center")
plt.xticks(tick_locations, prev_deaths_df_sorted["State"], rotation="vertical")
plt.title('Preventable Deaths per 100,000 Population per State (2018-2019)')
plt.xlabel('State')
plt.ylabel('Deaths per 100k Population')
plt.show()

In [ ]:
# COPY
# We ATTEMPTED to use the existing dataframes without the US -- but because those dropped based on index
# the indexes didn't align and we got errors -- going to try to redo dfs instead

# unis_clean_1819.reset_index()
# percentage_unis = unis_clean_1819["Percentage of Uninsured Adults"]
# unis_1819_df = pd.DataFrame(percentage_unis)
# unis_1819_df.head()

In [ ]:
# COPY
# avoi_deaths_df.reset_index()
# avoi_factor = avoi_deaths_df["Deaths per 100k Population"]
# avoi_clean_df = pd.DataFrame(avoi_factor)
# avoi_clean_df.head()

In [ ]:
# COPY
# Creating an official DataFrame for the uninsured data to utilize with graphs
unis_1819_steptodf = census_1819_df.groupby("State")["Percentage of Uninsured Adults"].mean().reset_index()
unis_factor = unis_1819_steptodf["Percentage of Uninsured Adults"]
unis_1819_df = pd.DataFrame(unis_factor)

# Display the resulting DataFrame
unis_1819_df.head()

In [ ]:
# COPY
# creating an official dataframe for the avoidable deaths data
avoi_factor = avoi_deaths_narrow["Deaths per 100k Population"]
avoidable_deaths_df = pd.DataFrame(avoi_factor)
avoidable_deaths_df.head()

In [ ]:
# COPY
uninsured = unis_1819_df['Percentage of Uninsured Adults']
deaths = avoidable_deaths_df['Deaths per 100k Population']
# the next line (pd.concat) is from ChatGPT : https://pandas.pydata.org/pandas-docs/stable/reference/api/pandas.concat.html
clean_data_df = pd.concat([uninsured, deaths], axis=1).dropna()
clean_data_df.head()

In [ ]:
# COPY
x_uninsured = clean_data_df.iloc[:,0] 
y_deaths = clean_data_df.iloc[:,1] 
correlation = st.pearsonr(x_uninsured,y_deaths) 
plt.scatter(x_uninsured,y_deaths) 
plt.xlabel('Percentage Uninsured') 
plt.ylabel('Avoidable Deaths per 100,000 Population')
plt.title('Scatter Plot Showing Correlation Between Uninsured Individuals and Avoidable Deaths')
print(f"The correlation between both factors is {round(correlation[0],2)}") 
plt.show()

In [ ]:
# COPY
unis_avoi = pd.merge(unis_clean_1819, avoi_deaths_df, on=["State"])
unis_avoi.head(15)

In [ ]:
# COPY
x_uninsured = unis_avoi.iloc[:,1] 
y_deaths = unis_avoi.iloc[:,3] 
correlation = st.pearsonr(x_uninsured,y_deaths) 
plt.scatter(x_uninsured,y_deaths) 
plt.xlabel('Percentage Uninsured') 
plt.ylabel('Avoidable Deaths per 100,000 Population')
plt.title('Scatter Plot Showing Correlation Between Uninsured Individuals and Avoidable Deaths')
print(f"The correlation between both factors is {round(correlation[0],2)}") 
plt.show()

In [ ]:
# COPY
unis_treat = pd.merge(unis_clean_1819, treat_deaths_df, on=["State"])
unis_treat.tail(15)

In [ ]:
# COPY
x_uninsured = unis_treat.iloc[:,1] 
y_deaths = unis_treat.iloc[:,3] 
correlation = st.pearsonr(x_uninsured,y_deaths) 
plt.scatter(x_uninsured,y_deaths) 
plt.xlabel('Percentage Uninsured') 
plt.ylabel('Treatable Deaths per 100,000 Population')
plt.title('Scatter Plot Showing Correlation Between Uninsured Individuals and Treatable Deaths')
print(f"The correlation between both factors is {round(correlation[0],2)}") 
plt.show()

In [ ]:
# COPY
unis_prev = pd.merge(unis_clean_1819, prev_deaths_df, on=["State"])
unis_prev.tail(15)

In [ ]:
# COPY

x_uninsured = unis_prev.iloc[:,1] 
y_deaths = unis_prev.iloc[:,3] 
correlation = st.pearsonr(x_uninsured,y_deaths) 
plt.scatter(x_uninsured,y_deaths) 
plt.xlabel('Percentage Uninsured') 
plt.ylabel('Preventable Deaths per 100,000 Population')
plt.title('Scatter Plot Showing Correlation Between Uninsured Individuals and Preventable Deaths')
print(f"The correlation between both factors is {round(correlation[0],2)}") 
plt.show()